In [ ]:
# to automatically reload modules who's content has changed
%load_ext autoreload
%autoreload 2

# configure matplotlib
%matplotlib inline
#%config InlineBackend.figure_format = 'svg'

In [ ]:
import time
import numpy as np
import GPy
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

import ipywidgets as widgets
from IPython.display import display
from IPython.core.debugger import set_trace

In [ ]:
import function_bo as fbo
from function_bo_plotting import *

import sys
sys.path.append('../')
from functions import Activations, WeightedBasisFunctions

In [ ]:
domain_bounds = ('x', 0, 10)
range_bounds = (-5, 5)

class Coordinator(fbo.Coordinator):
    def get_pre_phase_config(self, trial_num):
        c = fbo.GPPriorSelectConfig(self.domain_bounds)
        return c

    def get_bayes_config(self, trial_num):
        c = fbo.BayesSelectConfig(self.domain_bounds)
        c.tracking_l = 10
        return c


def make_objective(to_fit, sample_num, sample_dist):
    def objective(f):
        _, xmin, xmax = domain_bounds
        # global reward
        R_g = integrate(lambda x: (f(x) - to_fit(x))**2, (xmin, xmax))
        # local rewards
        R_ls = []
        if sample_dist == 'linear':
            reward_xs = np.linspace(xmin, xmax, num=sample_num)
        elif sample_dist == 'random':
            reward_xs = np.random.uniform(xmin, xmax, size=(sample_num,))
        else:
            raise ValueError()
        for x in reward_xs:
            R_l = (f(x)-to_fit(x))**2
            R_ls.append((x, R_l))
        return R_ls, R_g
    return objective

# set this global after each optimisation
op = None

def test_approx_function(to_fit, coordinator, objective=None):
    """
    Args:
        to_fit: the function to approximate
        trials: (pre_phase_trials, max_trials)
    """
    if objective is None:
        objective = make_objective(to_fit, 10, 'linear')

    np.random.seed(0)
    opt = fbo.Optimiser(objective, domain_bounds, range_bounds, desired_extremum='min', coordinator=coordinator)
    opt.run()
    plot_convergence(opt, best_R_g=0)
    plot_trials(opt, opt.trials, to_fit, color_by_reward=True)
    plot_surrogate_with_trials(opt, -1, to_fit)
    
    inc_i, inc = opt.get_incumbent()
    print('incumbent = trial {}'.format(inc_i))
    plot_trial_area(opt, inc, to_fit)
    
    global op
    op = opt

def plot_to_fit(to_fit):
    _, xmin, xmax = domain_bounds
    xs = np.linspace(xmin, xmax, num=100)
    plt.plot(xs, [to_fit(x) for x in xs], 'k--', label='to fit')
    plt.xlabel('x')
    plt.ylabel('y')
    plt.legend()
    plt.show()

In [ ]:
def to_fit(x):
    return np.sin(x) * 2*np.cos(x/4)
plot_to_fit(to_fit)

# Test fitting a simple function with a single attempt

In [ ]:
test_approx_function(to_fit, Coordinator(domain_bounds, 3, 4))

# Test fitting a simple function with a two attempts

In [ ]:
test_approx_function(to_fit, Coordinator(domain_bounds, 3, 5))

In [ ]:
plot_surrogate_3D(op, op.trials[-1].surrogate, flip_z=True)

# Test fitting with only a single random sample in the pre-phase

In [ ]:
op = test_approx_function(to_fit,  Coordinator(domain_bounds, 1, 10))

# Optimise a noisy function

In [ ]:
def to_fit_noisy(x):
    size = x.size if isinstance(x, np.ndarray) else None
    return to_fit(x) + np.random.normal(loc=0, scale=0.2, size=size)

plt.subplots(figsize=(16,8))
plot_to_fit(to_fit_noisy)

In [ ]:
test_approx_function(to_fit_noisy, Coordinator(domain_bounds, 4, 10))

In [ ]:
plot_surrogate_3D(op, op.trials[-4].surrogate, flip_z=True)

# Fit a function which has flat regions and more intricate detail

In [ ]:
f = WeightedBasisFunctions(arguments=[(2, 2, 1, 6), (6, 1, 1, 1), (8, 1, 2, 2)], # center, width, weight, power
                               functions=Activations.super_Gaussian)
def plot_super_gaussian():
    fig, ax = plt.subplots(figsize=(10, 8))
    _, xmin, xmax = domain_bounds
    xs = np.linspace(xmin, xmax, num=200)
    f.plot(ax, xs, color='r', show_basis=False, label='Super Gaussian')
plot_super_gaussian()

In [ ]:
test_approx_function(f, Coordinator(domain_bounds, 10, 15))

# Sample R_l more frequently

In [ ]:
_, xmin, xmax = domain_bounds
objective = make_objective(f, 20, 'linear')

class Coordinator2(fbo.Coordinator):
    def get_pre_phase_config(self, trial_num):
        c = fbo.GPPriorSelectConfig(self.domain_bounds)
        return c

    def get_bayes_config(self, trial_num):
        c = fbo.BayesSelectConfig(self.domain_bounds)
        c.surrogate_model_params = dict(
            kernel=GPy.kern.RBF(input_dim=2, ARD=True),
            normalizer=True
        )
        c.tracking_l = 10
        return c

test_approx_function(f, Coordinator2(domain_bounds, 5, 10), objective=objective)

In [ ]:
plot_surrogate_3D(op, op.trials[-4].surrogate, flip_z=True)

# Sampling even more frequently

In [ ]:
_, xmin, xmax = domain_bounds
objective = make_objective(f, 40, 'linear')

class CoordinatorSparse(fbo.Coordinator):
    def get_pre_phase_config(self, trial_num):
        c = fbo.GPPriorSelectConfig(self.domain_bounds)
        return c

    def get_bayes_config(self, trial_num):
        c = fbo.BayesSelectConfig(self.domain_bounds)
        c.surrogate_class = GPy.models.SparseGPRegression
        c.surrogate_model_params = dict(
            kernel = GPy.kern.RBF(input_dim=2),
            num_inducing=20,
            normalizer=True
        )
        c.surrogate_optimise_params = dict(
            parallel = False, # Can't use parallel optimisation with sparse GP (bug)
            verbose = True,
            num_restarts = 1
        )
        c.tracking_l = 10
        return c

test_approx_function(f, CoordinatorSparse(domain_bounds, 10, 20), objective=objective)

# Investigate the effect of sampling R_l at different places each trial
Also in the pre_phase the sampled functions are engineered to hopefully be more representative of the function to fit (non-zero mean and variance > 1) 

In [ ]:
_, xmin, xmax = domain_bounds
objective = make_objective(f, 10, 'random')

class Coordinator3(fbo.Coordinator):
    def get_pre_phase_config(self, trial_num):
        c = fbo.GPPriorSelectConfig(self.domain_bounds)
        c.mu = lambda x: 1.0 # bias
        c.kernel = GPy.kern.RBF(input_dim=1, variance=1.5, lengthscale=1.0) # TODO: not multi-dimensional
        return c

    def get_bayes_config(self, trial_num):
        c = fbo.BayesSelectConfig(self.domain_bounds)
        return c

test_approx_function(f, Coordinator3(domain_bounds, 10, 15), objective=objective)

In [ ]:
plot_surrogate_3D(op, op.trials[-4].surrogate, flip_z=True)